In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master('local[*]')
    .appName('PySparkShell')
    .getOrCreate()
)

In [2]:
import sys
import os

sys.path.append('/dbfs/cdr/library')

from pandas import set_option
set_option('display.max.columns', None)

# TODO: remove for Databricks
from dbutils import DBUtils, FileInfo
dbutils = DBUtils()

In [3]:
from typing import Dict, List, Optional

# from sparkFunctions import create_tmp_parquet

# adds file system prefix /dbfs for local and dbfs: for databricks
def resolve(path: str) -> Optional[str]:
    try:
        prefix = dbutils.fs.ls('/')[0].path[:5]
        return prefix + path
    except Exception as error:
        print(error)

In [4]:
FOLDER = '/automation'

ZOOM_FOLDER = os.path.join(FOLDER, 'ZoomData')
BASE_FOLDER = os.path.join(FOLDER, 'OnBase')

In [5]:
[file.name for file in dbutils.fs.ls(ZOOM_FOLDER)]

['AccountCallLogs.json',
 'formattedAccountCallLogs.parquet/',
 'formattedAccountCallLogsTmp.parquet/']

In [94]:
path = os.path.join(ZOOM_FOLDER, 'formattedAccountCallLogs.parquet')

calls = spark.read.parquet(resolve(path))
calls.limit(5).toPandas()

,caller_name,caller_number,callee_name,callee_number,direction,duration,date_time,result,call_id,extension_number,id,name,type,call_type,caller_number_type,callee_number_type,path,has_recording,has_voicemail,date_time_est
0,Ashton Sower,409,Halfpenny Techn,+16102779100,outbound,75,2021-08-30T21:05:22Z,Call connected,7002332237789502144,409,PBO-YsLMTf2ncnsYKSuGvQ,Ashton Sower,user,pstn,1,2,pstn,False,False,2021-08-30 17:05:22.000
1,Annie Lewis,275,None,+18888958106,outbound,32,2021-08-30T21:05:35Z,Call connected,7002332293623987563,275,ukEdcYAiTSyjVGTPmdH5GQ,Annie Lewis,user,pstn,1,2,pstn,False,False,2021-08-30 17:05:35.000
2,Annie Lewis,275,None,+18189515300,outbound,6,2021-08-30T21:07:15Z,Call connected,7002332723120795534,275,ukEdcYAiTSyjVGTPmdH5GQ,Annie Lewis,user,pstn,1,2,pstn,False,False,2021-08-30 17:07:15.000
3,Dulce Reyes-Loredo,228,Jubilantradioph,+14074556732,outbound,9,2021-08-30T21:09:12Z,Call connected,7002333225631927388,228,zIt5Z6ZzTWG9wHmZpBkASg,Dulce Reyes-Loredo,user,pstn,1,2,pstn,False,False,2021-08-30 17:09:12.000
4,Michelle Regnier,541,Augusta Health,+15409324555,outbound,64,2021-08-30T21:10:28Z,Call connected,7002333547754475798,541,4GM2Dtm-Tnea8Jzhj3QaEw,Michelle Regnier,user,pstn,1,2,pstn,False,False,2021-08-30 17:10:28.000


In [34]:
# calls.printSchema()

In [68]:
from pyspark.sql.functions import col, length, regexp_replace

# calls.select('callee_number').show()

# only numbers, + and *
# calls.where(col('callee_number').rlike(r'[^0-9+\*]')).count()

calls = (
    calls
    .withColumn('PHONE_NUMBER', regexp_extract('callee_number', r'^(\+1|\*)*([0-9]+)', 2).cast('long'))
    # .select('callee_number', 'PHONE_NUMBER')
    # .withColumn('PHONE_NUMBER', regexp_replace('callee_number', '[^0-9]', '').cast('long'))
    # .withColumn('_LENGTH', length('PHONE_NUMBER'))
    # .where('_LENGTH > 12')
    # .where("callee_number LIKE '%*%'")
    # .toPandas()
    # .groupBy('_LENGTH')
    # .count()
    # .show()
    # .select('PHONE_NUMBER', '_LENGTH')
    # .show()
    # .withColumn('PHONE_NUMERIC', regexp_extract('PHONE_NUMERIC', r'^(\d{0,10})', 1))
)

In [96]:
from pyspark.sql.functions import desc, explode, split

# (
#     calls
#     .withColumn('_PHONE', regexp_replace('caller_number', '[0-9]', ' '))
#     .withColumn('_SPLIT', split('_PHONE', r'\s+'))
#     .withColumn('_EXPLODE', explode('_SPLIT'))
#     .groupBy('_EXPLODE')
#     .count()
#     .sort(desc('count'))
#     .show(100)
# )

# calls.where("CALLER_NUMBER LIKE '%%'").toPandas()

,caller_name,caller_number,callee_name,callee_number,direction,duration,date_time,result,call_id,extension_number,id,name,type,call_type,caller_number_type,callee_number_type,path,has_recording,has_voicemail,date_time_est


In [9]:
[file.name for file in dbutils.fs.ls(BASE_FOLDER)]

['rm_DVStatementRequestActivityRecords.parquet',
 'rm_DVStatementRequests.parquet',
 'rm_DVVendorContacts.parquet']

In [10]:
path = os.path.join(BASE_FOLDER, 'rm_DVStatementRequestActivityRecords.parquet')

activities = spark.read.parquet(resolve(path))
activities.limit(5).toPandas()

,ObjectID,CreatedDate,ReferenceNumber,CustomerVendorName,JobNumber,JobName,ContactType,ActivityUser,ActivityDate,FollowUpDate,STNID,Notes,Outcome,ActivityType,CustVendorObjectID,VendorContactObjectID,StatementRequestObjectID,CreatedBy
0,28785475,2020-11-02 06:41:33.170,1279250,ULINE ...,3412,Carolinas Shared Service - 3412 ...,Call,ASIBLEY,2020-11-01 19:00:00,2020-11-03 19:00:00,NaN,This is a test - Updated,None,None,21259327,NaN,25422522,ASIBLEY
1,28785637,2020-11-02 06:47:27.003,1279250,ULINE ...,3412,Carolinas Shared Service - 3412 ...,Email,ASIBLEY,2020-11-01 19:00:00,2020-11-04 19:00:00,NaN,Test 2 - Updated,None,None,21259327,NaN,25422522,ASIBLEY
2,28786094,2020-11-02 06:52:58.307,1279250,ULINE ...,3412,Carolinas Shared Service - 3412 ...,Client Email,ASIBLEY,2020-11-01 19:00:00,2020-11-03 19:00:00,NaN,Test 3,None,None,21259327,NaN,25422522,ASIBLEY
3,28838136,2020-11-04 12:47:58.533,1287154,BIOCOMPOSITES INC. ...,3435,The Christ Hospital - 3435 ...,None,bwilliams1,2020-11-03 19:00:00,NaT,6192802.0,Statement Request creation skipped because thi...,None,None,27542411,NaN,27781594,ASIBLEY
4,28838138,2020-11-04 12:48:02.617,1225130,BONA FIDE COMMERCIAL SERVICES ...,3177,UC Health - 3177 ...,None,jdagher,2020-11-03 19:00:00,NaT,6192806.0,Statement Request creation skipped because thi...,None,None,24317597,NaN,25860124,ASIBLEY


In [93]:
activities.groupBy('ActivityType').count().show(truncate=False)

+-------------------------+-------+
|ActivityType             |count  |
+-------------------------+-------+
|Called Vendor            |98457  |
|Received Call / Email    |4075   |
|null                     |1491895|
|Note Only                |25191  |
+-------------------------+-------+



In [15]:
# activities.printSchema()

In [12]:
path = os.path.join(BASE_FOLDER, 'rm_DVStatementRequests.parquet')

requests = spark.read.parquet(resolve(path))
requests.limit(5).toPandas()

,JobNo,JobTier,CustomerName,CustVendorID,VendorNo,CustVendorGroupID,WNC,StatementWNC,VendorGroupName,Volume,RequestDate,ReferenceNumber,Status,RequestMethod,RequestType,Contact,RequesterFullName,RequestText,LastActivityDate,LastStatementReceivedDate,CreatedDate,CallsheetNo,ObjectID,CallerStatus,ReconStatus,CurrentAssigneeID,CurrentAssigneeName,EnteredReconDate,LastReconQueueName,LastReconQueueEntryDate,AccountsReceived,AccountsRequested,MessageID,VendorContactObjectID
0,3427,NaN,New Hanover Regional Medical Center ...,415960,16845 ...,415959,NaN,NaN,A M COFFEE DISTRIBUTORS LLC ...,0.00,2021-09-02 20:00:00,1285660,No Receipt,MassEmail,Caller,amcoffeedist@gmail.com ...,Cindy Allen ...,None,2021-08-22 20:00:00,NaT,2021-09-02 22:50:28.007,C-338914,36846273,Sent Authorization Letter ...,None,8398263,Rebekah Dykema ...,NaT,None,NaT,NaN,NaN,hONrG5xpTUyiQB4Rs7gb0g ...,29510277.0
1,3427,NaN,New Hanover Regional Medical Center ...,415960,16845 ...,415959,NaN,NaN,A M COFFEE DISTRIBUTORS LLC ...,0.00,2020-07-23 20:00:00,1285660,Superceded,MassEmail,None,amcoffeedist@gmail.com ...,Jessica Rinehart ...,None,2020-07-23 20:00:00,NaT,2020-07-24 12:29:52.840,C-338914,26681846,None,None,19272454,Yodit Kahssai ...,NaT,None,NaT,NaN,NaN,None,NaN
2,3427,NaN,New Hanover Regional Medical Center ...,415960,16845 ...,415959,NaN,NaN,A M COFFEE DISTRIBUTORS LLC ...,0.00,2020-07-30 20:00:00,1285660,Superceded,MassEmail,Mass 1,amcoffeedist@gmail.com ...,Jessica Rinehart ...,None,2020-07-30 20:00:00,NaT,2020-07-31 04:05:15.817,C-338914,26819776,None,None,19272454,Yodit Kahssai ...,NaT,None,NaT,NaN,NaN,None,NaN
3,3427,NaN,New Hanover Regional Medical Center ...,415960,16845 ...,415959,NaN,NaN,A M COFFEE DISTRIBUTORS LLC ...,0.00,2021-01-18 19:00:00,1285660,Superceded,MassEmail,Caller,amcoffeedist@gmail.com ...,Jessica Rinehart ...,None,2020-07-30 20:00:00,NaT,2021-01-19 07:01:21.287,C-338914,30690711,Sent Authorization Letter ...,None,8398263,Rebekah Dykema ...,NaT,None,NaT,NaN,NaN,None,29510277.0
4,3427,NaN,New Hanover Regional Medical Center ...,415960,16845 ...,415959,NaN,NaN,A M COFFEE DISTRIBUTORS LLC ...,0.00,2021-08-18 20:00:00,1285660,Superceded,MassEmail,Mass 1,amcoffeedist@gmail.com ...,Cindy Allen ...,None,2021-08-22 20:00:00,NaT,2021-08-22 20:03:52.047,C-338914,36365526,Sent Authorization Letter ...,None,8398263,Rebekah Dykema ...,NaT,None,NaT,NaN,NaN,hONrG5xpTUyiQB4Rs7gb0g ...,29510277.0


In [14]:
# requests.printSchema()

In [69]:
path = os.path.join(BASE_FOLDER, 'rm_DVVendorContacts.parquet')

vendors = spark.read.parquet(resolve(path))
vendors.limit(5).toPandas()

,CustVendorGroupNo,CustVendorNo,ContactType,TypePreferredContact,LastName,FirstName,FullName,Email,Phone,Fax,Title,Note,ExternalID,BSAPVCID,CID,CustomerName,CVObjectID,CustObjectID,ObjectID,CreatedDate,CreatedBy,BSAPVendorObjectID
0,68983,6472 ...,Accounts Receivable,NaN,Morris ...,Rita ...,Rita Morris ...,rita.morris@navigant.com ...,7049044725,None,None,Contact was featured on a letter that was incl...,None,NaN,2959,New Hanover Regional Medical Center ...,454698,291094,556858,2016-08-02 08:40:45.847,PVANDERBERG,556858
1,68983,6472 ...,Accounts Receivable,NaN,None,None,None,nci_apinvoices@navigant.com ...,None,None,None,Corporate Accounting email address retrieved f...,None,NaN,2959,New Hanover Regional Medical Center ...,454698,291094,559542,2016-08-02 08:55:05.970,PVANDERBERG,559542
2,70348,4332 ...,Accounts Receivable,NaN,None,None,None,ar@pharmedium.com ...,8005237749,None,None,Credit & Collections Department contact retrie...,None,NaN,2959,New Hanover Regional Medical Center ...,459237,291094,610390,2016-08-02 12:21:38.310,PVANDERBERG,610390
3,70348,4332 ...,Accounts Receivable,NaN,Ohlson ...,Lee ...,Lee Ohlson ...,lohlson@pharmedium.com ...,8474572340,None,Credit & Collections Lead ...,None,None,NaN,2959,New Hanover Regional Medical Center ...,459237,291094,611761,2016-08-02 12:26:48.133,PVANDERBERG,611761
4,61176,2509 ...,Accounts Receivable,NaN,Beeghly ...,Colleen ...,Colleen Beeghly ...,Cbeeghly@fffenterprises.com ...,(800) 843-7477 x1280,None,Credit & Collections Coordinator ...,Contact retrieved from reference #934518 from ...,None,NaN,2959,New Hanover Regional Medical Center ...,436103,291094,953252,2016-08-03 04:47:50.640,PVANDERBERG,953252


In [32]:
# from pyspark.sql.functions import desc, explode, split

# (
#     vendors
#     .withColumn('_PHONE', regexp_replace('Phone', '[0-9]', ' '))
#     .withColumn('_SPLIT', split('_PHONE', r'\s+'))
#     .withColumn('_EXPLODE', explode('_SPLIT'))
#     .groupBy('_EXPLODE')
#     .count()
#     .sort(desc('count'))
#     .show(100)
# )

In [70]:
from pyspark.sql.functions import col, regexp_extract, trim, when

vendors = (
    vendors
    .withColumn('EXTENSION', regexp_extract(trim('Phone'), r'(?<=[^0-9\-\.])+([0-9]+)$', 1))
    .withColumn('EXTENSION', col('EXTENSION').cast('long'))
    # .withColumn('EXTENSION', when(col('EXTENSION') != '', col('EXTENSION')))
    # .where('EXTENSION IS NOT NULL')
    #.where(col('EXTENSION').rlike('^0'))
    # .select('PHONE', 'EXTENSION')
    # .write
    # .csv('/tmp/VENDORS')
    # .show(100, truncate=False)
    # .count()
)

In [71]:
from pyspark.sql.functions import regexp_extract, regexp_replace

vendors = (
    vendors
    .withColumn('PHONE_NUMBER', regexp_replace('Phone', '[^0-9]', ''))
    .withColumn('PHONE_NUMBER', regexp_extract('PHONE_NUMBER', r'^(\d{0,10})', 1).cast('long'))
    # .select('PHONE', 'PHONE_NUMERIC', '_PHONE')
    # .show(truncate=False)
)

In [84]:
(
    vendors
    .join(
        calls,
        # on=vendors['PHONE_NUMERIC'].contains(calls['PHONE_NUMERIC']),
        on=['PHONE_NUMBER'],
        how='left_semi'
    )
    .count()
)

81796

In [75]:
calls.createOrReplaceTempView(name='CALLS')
vendors.createOrReplaceTempView(name='VENDORS')

In [81]:
# query = """
# SELECT
#     CALLS.*
# FROM
#     CALLS
# WHERE
#     EXISTS (
#         SELECT 1
#         FROM
#             VENDORS
#         WHERE
#             instr(VENDORS.PHONE_NUMBER, CALLS.PHONE_NUMBER) > 0
#             -- CALLS.PHONE_NUMBER LIKE VENDORS.PHONE_NUMBER
#             -- CALLS.EXTENSION_NUMBER = VENDORS.EXTENSION
#     )
# """

# table = spark.sql(query)
# table.count()

In [89]:
# calls.groupBy('EXTENSION_NUMBER').count().show()

In [74]:
calls.count()

43405